## Imports

In [1]:
import gym
# import gym_minigrid
from gym_minigrid.wrappers import *
import time
import pickle
from os.path import exists
# import random
import functools
import numpy as np
import pandas as pd
import hashlib
import time
import logging
import matplotlib.pyplot as plt
# import typing

logging.basicConfig(format='%(asctime)s - %(message)s', level=logging.INFO)


def timer(func):
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        logging.debug(f"Starting {func.__name__!r}.")
        start_time = time.perf_counter()
        value = func(*args, **kwargs)
        end_time = time.perf_counter()
        run_time = end_time - start_time
        logging.debug(f"Finished {func.__name__!r} in {run_time:.4f} secs.")
        return value
    return wrapper_timer

In [2]:
@timer
def reshape_state(state: np.ndarray) -> np.ndarray:
    (rows, cols, x) = state.shape
    _ = np.reshape(state, [rows*cols*x, 1], 'F')[0:rows*cols]
    state_reshaped = np.reshape(_, [rows, cols], 'F')
    return state_reshaped


@timer
def generate_state_hash(arr: np.ndarray) -> int:
    arr_enc = str(arr).encode()
    return hashlib.md5(arr_enc).hexdigest()


@timer
def initialise_env(env_vars) -> np.ndarray:
    env = gym.make(env_vars['ENV_NAME'])
    env = ImgObsWrapper(env)
    return env


@timer
def initialise_state(env):
    _ = env.reset()
    _state = reshape_state(_)
    state = generate_state_hash(_state)
    return state


@timer
def take_random_action(env_vars):
    """
    left = 0
    right = 1
    forward = 2
    """
    return np.random.randint(0, env_vars['possible_actions'])


@timer
def take_action_epsilon_greedy(q_table, state, env_vars, epsilon):
    if (np.random.random() < epsilon):
        action = take_random_action(env_vars)
    else:
        action = np.argmax(q_table[state])
    logging.debug(f"Action chosen: {action}")
    return action


@timer
def initialise_q(env_vars, state):
    q = {}
    val = np.full((env_vars['possible_actions'],), env_vars['init_q_val'])
    q[state] = val
    return q


def create_grid(env_vars):
    grid = {}
    key = 0
    for i in range(len(env_vars['alpha_range'])):
        for j in range(len(env_vars['gamma_range'])):
            grid[key] = {'alpha': env_vars['alpha_range'][i],
                         'gamma': env_vars['gamma_range'][j]}
            key = key + 1
    logging.info(f"Grid: {grid}")
    return grid


def update_epsilon(env_vars: dict, prev_epsilon: float, episode: int) -> float:
    if prev_epsilon > env_vars['min_epsilon']:
        epsilon = prev_epsilon * env_vars['epsilon_decay_rate']
    else:
        epsilon = env_vars['min_epsilon']
    return epsilon


@timer
def train(env_vars, hparams):
    env = initialise_env(env_vars)
    state = initialise_state(env)
    q = initialise_q(env_vars, state)
    rewards = []
    step_tracker = []
    prev_epsilon = env_vars['max_epsilon']  # Initalise epsilon to max_value
    for episode in range(env_vars['EPISODES']):
        state = initialise_state(env)
        epsilon = update_epsilon(env_vars, prev_epsilon, episode)
        for step in range(0, env_vars['MAX_STEPS']):
            action = take_action_epsilon_greedy(q, state, env_vars, epsilon)
            _next_state, reward, done, info = env.step(action)
#             env.render() # render the environment, this does not work inside Jupyter notebook
#             time.sleep(0.05) # When training without rendering remove this line
            if done:
                if reward > 0:
                    step_tracker.append(step)
                    logging.debug(
                        f"Finished episode successfully taking {step} steps and receiving reward {reward}")
                else:
                    pass
                break
            _next_state = reshape_state(_next_state)
            next_state = generate_state_hash(_next_state)
            if q.get(next_state) is None:
                val = np.full(
                    (env_vars['possible_actions'],), env_vars['init_q_val'])
                q[next_state] = val
            next_action = np.argmax(q[next_state])
            q[state][action] = q[state][action] + hparams['alpha'] * \
                (reward + hparams['gamma'] * (q[next_state]
                 [next_action]) - q[state][action])
            state = next_state
        rewards.append(reward)
        prev_epsilon = epsilon  # Update epsilon so that it can decay
    agent_name = f'agent_{int(time.time())}'
    _save_agent(env_vars, q, agent_name)
    plot_reward(env_vars, rewards, agent_name)
    result = {'alpha': hparams['alpha'],
              'gamma': hparams['gamma'],
              'min_epsilon': env_vars['min_epsilon'],
              'max_epsilon': env_vars['max_epsilon'],
              'epsilon_decay_rate': env_vars['epsilon_decay_rate'],
              'avg_reward': np.sum(rewards) / np.count_nonzero(rewards),
              'completion_rate': np.count_nonzero(rewards) / len(rewards),
              'avg_steps': np.sum(step_tracker) / len(step_tracker)}
    return agent_name, result


@timer
def _save_agent(env_vars, agent, agent_name):
    with open(f"{env_vars['model_dir']}/{agent_name}.pickle", 'wb') as handle:
        pickle.dump(agent, handle, protocol=pickle.HIGHEST_PROTOCOL)
        logging.info(f"Q table saved successfully.")
        handle.close()


@timer
def load_agent(env_vars):
    with open(f"{env_vars['filename']}", 'rb') as handle:
        q_table = pickle.load(handle)
        handle.close()
    return q_table


@timer
def plot_reward(env_vars, rewards, agent_name):
    fig, ax = plt.subplots()
    fig.set_size_inches(8, 8)
    ax.set_title(fr"Accumalated Reward averaged over time")
    ax.set_xlabel('Iterations')
    ax.set_ylabel('Accumulated Reward')
    x = [i+1 for i in range(len(rewards))]
    acc_rewards = np.cumsum(rewards) / len(rewards)
    ax.plot(x, acc_rewards)
    plt.savefig(f"{env_vars['perf_art_dir']}/reward_{agent_name}.png")
    logging.info(f"Rewards plot saved successfully.")
    plt.close(plt.gcf())


@timer
def write_results(env_vars, results):
    df = pd.DataFrame.from_dict(results, orient='index')
    df.to_csv(f"{env_vars['gs_results']}/gs_results_{int(time.time())}.csv")
    logging.info(f"Grid search results csv saved successfully.")


def main():
    env_vars = {}
    env_vars['ENV_NAME'] = 'MiniGrid-Empty-8x8-v0'
    env_vars['EPISODES'] = 50
    env_vars['MAX_STEPS'] = 8 * 8 * 4  # Grid size times possible action states
    env_vars['possible_actions'] = 3
    env_vars['init_q_val'] = 0
    env_vars['model_dir'] = '../data/model-file'
    env_vars['gs_results'] = '../data/grid-search-results'
    env_vars['perf_art_dir'] = '../data/performance-artefacts'
    env_vars['max_epsilon'] = 0.99
    env_vars['min_epsilon'] = 0.05
    env_vars['epsilon_decay_rate'] = 0.999
    env_vars['alpha_range'] = np.arange(0.1, 1, 0.1)
    env_vars['gamma_range'] = np.arange(0.1, 1, 0.1)
    results = {}
    grid = create_grid(env_vars)
    for key in grid.keys():
        hparams = grid[key]
        agent_name, result = train(env_vars, hparams)
        results[agent_name] = result
    write_results(env_vars, results)

In [3]:
def predict(env_vars, q):
    env = initialise_env(env_vars)
    state = initialise_state(env)
    rewards = []
    step_tracker = []
    epsilon = 0
    for episode in range(env_vars['EPISODES']):
        state = initialise_state(env)
        for step in range(0, env_vars['MAX_STEPS']):
            action = np.argmax(q[state])
            _next_state, reward, done, info = env.step(action)
#             env.render() # render the environment, this does not work inside Jupyter notebook
#             time.sleep(0.05) # When training without rendering remove this line
            if done:
                if reward > 0:
                    step_tracker.append(step)
                    logging.debug(
                        f"Finished episode successfully taking {step} steps and receiving reward {reward}")
                else:
                    pass
                break
            _next_state = reshape_state(_next_state)
            next_state = generate_state_hash(_next_state)
            state = next_state
        rewards.append(reward)
#     agent_name = f'agent_{int(time.time())}'
#     plot_reward(env_vars, rewards, agent_name)
    result = {'avg_reward': np.sum(rewards) / np.count_nonzero(rewards),
              'completion_rate': np.count_nonzero(rewards) / len(rewards),
              'avg_steps': np.sum(step_tracker) / len(step_tracker)}
    return result, rewards, step_tracker

In [4]:
def load_agent(env_vars):
    with open(f"{env_vars['best_agent_filename']}", 'rb') as handle:
        q_table = pickle.load(handle)
        handle.close()
    return q_table

In [21]:
env_vars = {}
env_vars['ENV_NAME'] = 'MiniGrid-Empty-8x8-v0'
env_vars['best_agent_filename'] = '../data/model-file/agent_1658092802.pickle'
env_vars['EPISODES'] = 2000
env_vars['MAX_STEPS'] = 8 * 8 * 4

In [22]:
env_vars['best_agent_filename'] = '../data/model-file/agent_1658092802.pickle'

In [23]:
q = load_agent(env_vars)

In [24]:
q

{'832b9c9e9096b18a959bb7d11b6deea5': array([0.24041154, 0.25731782, 0.29622597]),
 'ce654dffc469a1d91dc6846f9f65c3ab': array([0.24357673, 0.26695036, 0.24940709]),
 '2242cede2e03f22e795c2a5488d4299c': array([0.26316128, 0.25731725, 0.25844189]),
 'e975af2b2f4c4b7fd37227faad9776d2': array([0.27845848, 0.24836322, 0.29000982]),
 '9953dd082f2761210ca8ef5f182ef6a2': array([0.30514091, 0.31090208, 0.33016936]),
 'c9d88f03aa6563edf66866667d4e2e92': array([0.31801344, 0.3301441 , 0.31439572]),
 '7284cba00089ee4a0783e0756f459170': array([0.30587305, 0.40308968, 0.25721711]),
 '2193f1ec89a69bfaa93a65c8c63fffda': array([0.26751344, 0.29476937, 0.32968619]),
 '42205e33c5599bddf4cfedb2080ffab8': array([0.32896763, 0.3551578 , 0.36349246]),
 '68f29cbb3ed40a1c4e849c9fa19f0a3f': array([0.31987434, 0.33149024, 0.30477114]),
 '569d5d92444f66632515d625b4d21ab7': array([0.35486347, 0.32155472, 0.28038422]),
 '8f7b7c86d4d05191ffa41be547db9b2a': array([0.28942195, 0.28621528, 0.25907165]),
 '53af52a7078e3e

In [20]:
q

{'832b9c9e9096b18a959bb7d11b6deea5': array([0.24041154, 0.25731782, 0.29622597]),
 'ce654dffc469a1d91dc6846f9f65c3ab': array([0.24357673, 0.26695036, 0.24940709]),
 '2242cede2e03f22e795c2a5488d4299c': array([0.26316128, 0.25731725, 0.25844189]),
 'e975af2b2f4c4b7fd37227faad9776d2': array([0.27845848, 0.24836322, 0.29000982]),
 '9953dd082f2761210ca8ef5f182ef6a2': array([0.30514091, 0.31090208, 0.33016936]),
 'c9d88f03aa6563edf66866667d4e2e92': array([0.31801344, 0.3301441 , 0.31439572]),
 '7284cba00089ee4a0783e0756f459170': array([0.30587305, 0.40308968, 0.25721711]),
 '2193f1ec89a69bfaa93a65c8c63fffda': array([0.26751344, 0.29476937, 0.32968619]),
 '42205e33c5599bddf4cfedb2080ffab8': array([0.32896763, 0.3551578 , 0.36349246]),
 '68f29cbb3ed40a1c4e849c9fa19f0a3f': array([0.31987434, 0.33149024, 0.30477114]),
 '569d5d92444f66632515d625b4d21ab7': array([0.35486347, 0.32155472, 0.28038422]),
 '8f7b7c86d4d05191ffa41be547db9b2a': array([0.28942195, 0.28621528, 0.25907165]),
 '53af52a7078e3e

In [13]:
q = load_agent(env_vars)
len(q.keys())

86

In [14]:
result, rewards, step_tracker = predict(env_vars, q)

In [15]:
result

{'avg_reward': 0.9578124999999997, 'completion_rate': 1.0, 'avg_steps': 11.0}

In [16]:
rewards

[0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9578125,
 0.9

In [17]:
step_tracker

[11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,


In [11]:
# if __name__ == '__main__':
#     main()

3 metrics to judge agent:
average reward
number of time agent reaches goal - completion rate
number of steps to reach goal

alpha -> learning rate, make sure it isn't too high -> consider when do you anneal and what point? (not important) they use 0.7
alpha below 0.1 is more correct, it just requires a lot more training episodes. 
gamma -> can make it fairly large, can even make it 1. He used 0.65 or something. Depends on learning rate as well. 
Make the number of episodes much larger so it can truly learn. E.g. 5000 or 10000